# ITS_LIVE: Extract velocity map data at GNSS locations

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glaft.georaster import Raster

In [3]:
import rasterio
# import matplotlib.pyplot as plt
# from rasterio.plot import show as rio_show
# import matplotlib as mpl
import pandas as pd
import geopandas as gpd
import numpy as np
# import glob
from datetime import datetime
from pyproj import Transformer

In [4]:
df = pd.read_csv('../results_ITSLIVE.csv', dtype=str)
df

,Label,Start date,End date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,Software,Vx,Vy,Assigned-x-error,Assigned-y-error,SAV-uncertainty-x,SAV-uncertainty-y,SAV-peak-x,SAV-peak-y,SAV-outlier-percent
0,LS8-20180304-20180405,20180304,20180405,32,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,47.7,32.6,54.6898472387492,44.43550088148372,-10.03659591652361,12.709057726210915,24.234743614739713
1,LS8-20180405-20180421,20180405,20180421,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,119.6,71.7,139.11564778386023,121.72619181087768,37.1262759392107,-9.957908020263119,26.92653428127809
2,LS8-20180421-20180523,20180421,20180523,32,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,92.5,48.2,136.50749745725778,81.16662010972085,6.2045264085221845,-4.689391823169132,16.207432527907308
3,LS8-20180523-20180608,20180523,20180608,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,75.5,64.0,84.6755005418857,71.44495358221604,7.938328175801786,-10.938328175801757,25.96590909090909
4,LS8-20180412-20180428,20180412,20180428,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,132.2,68.8,103.47791768487576,96.80192299552894,46.36598141271389,-31.662002655326575,22.752202094066813
5,LS8-20180428-20180802,20180428,20180802,96,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,32.0,18.5,32.36846681024471,34.60077486612366,-29.34846208381842,1.6515379161815815,23.938584779706275
6,LS8-20180802-20180818,20180802,20180818,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,48.3,44.9,40.35395714351298,35.30971250057386,-20.783183482204336,-1.2610611607347835,30.545367564981664
7,LS8-20180818-20180903,20180818,20180903,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,49.9,47.6,43.38566683970623,43.385666839706225,0.19954436946062515,-3.8766777374018773,25.41811698785975
8,LS8-20180903-20181005,20180903,20181005,32,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,38.7,30.8,37.213387774984334,31.39879593514303,-14.814591839841306,4.83708163203174,31.19185192439526
9,Sen2-20180306-20180316,20180306,20180316,10,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),ITS_LIVE (autoRIFT),/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,/home/jovyan/Projects/PX_comparison/PX/ITS_LIV...,102.3,62.4,100.22543238091087,84.56520857139354,44.92431333332425,5.132044761903472,24.211788896504455


In [5]:
# All GNSS coordinates are in EPSG 32607 and have to be reprojected to EPSG3413 before sampling the geotiffs.
transformer = Transformer.from_crs("epsg:32607", "epsg:3413")

In [6]:
GPS_root = '/home/jovyan/Projects/PX_comparison/GPS/'

for idx, row in df.iterrows():
    startdate = datetime.strptime(row['Start date'], '%Y%m%d')
    enddate = datetime.strptime(row['End date'], '%Y%m%d')
    timedel = enddate - startdate
    duration = timedel.days / 365     # in yrs
    startdate_gpsstr = startdate.strftime('%Y-%m-%d')
    enddate_gpsstr = enddate.strftime('%Y-%m-%d')
    gps_file = GPS_root + 'Kaskawulsh_{}_to_{}_GPS'.format(enddate_gpsstr, startdate_gpsstr)
    
    gps = pd.read_csv(gps_file)
    ######## Prep corrdinates in EPSG3413 
    for idx2, row2 in gps.iterrows():
        x32607 = row2.start_easting
        y32607 = row2.start_northing
        x3413, y3413 = transformer.transform(x32607, y32607)
        gps.loc[idx2, 'start_easting_3413'] = x3413
        gps.loc[idx2, 'start_northing_3413'] = y3413
        x32607 = row2.end_easting
        y32607 = row2.end_northing
        x3413, y3413 = transformer.transform(x32607, y32607)
        gps.loc[idx2, 'end_easting_3413'] = x3413
        gps.loc[idx2, 'end_northing_3413'] = y3413
    #########
    # This is beginning coordinates
    gps = gpd.GeoDataFrame(gps, geometry=gpd.points_from_xy(gps['end_easting_3413'], gps['end_northing_3413']), crs='EPSG:3413')
    gps_xy = list(gps[['end_easting_3413', 'end_northing_3413']].to_records(index=False))
    
    gps['vx (m/yr)']  = (gps['start_easting_3413'] - gps['end_easting_3413']) / duration
    gps['vy (m/yr)']  = (gps['start_northing_3413'] - gps['end_northing_3413']) / duration
    gps['v (m/yr)']  = np.abs(gps['velocity (m/d)'] * 365)
    
    vx_grid = Raster(row.Vx)
    vy_grid = Raster(row.Vy)
    v_grid = Raster(row.Vx.replace('vx', 'v'))
    verr_grid = Raster(row.Vx.replace('vx', 'v_error'))
    sampled = []
    sampled2 = []
    for x, y in gps_xy:
        # print(gps_file, x, y)
        if np.isnan(x) or np.isnan(y) or np.isinf(x) or np.isinf(y):
            sampled.append([np.nan, np.nan, np.nan, np.nan])
            sampled2.append([np.nan, np.nan])
        else:
            vx_avg, vx_3by3 = vx_grid.value_at_coords(x, y, window=3, return_window=True)
            vy_avg, vy_3by3 = vy_grid.value_at_coords(x, y, window=3, return_window=True)
            vx_3by3[vx_3by3 < -9998] = np.nan   # ITSLIVE nodata = -32767
            vy_3by3[vy_3by3 < -9998] = np.nan   # ITSLIVE nodata = -32767
            vx_nn = vx_3by3[0, 1, 1]     # nearest neighbor value
            vy_nn = vy_3by3[0, 1, 1]
            if np.any(~np.isnan(vx_3by3)):
                vx_avg = np.nanmean(vx_3by3)
            else:
                vx_avg = np.nan
            if np.any(~np.isnan(vy_3by3)):
                vy_avg = np.nanmean(vy_3by3)
            else:
                vy_avg = np.nan

            # vx_nn -= float(df.loc[idx, 'SAV-peak-x'])
            # vx_avg -= float(df.loc[idx, 'SAV-peak-x'])
            # vy_nn -= float(df.loc[idx, 'SAV-peak-y'])
            # vy_avg -= float(df.loc[idx, 'SAV-peak-y'])

            sampled.append([vx_nn, vx_avg, vy_nn, vy_avg])
            
            v_avg, v_3by3 = v_grid.value_at_coords(x, y, window=3, return_window=True)
            v_3by3[v_3by3 < -9998] = np.nan   # ITSLIVE nodata = -32767
            if np.any(~np.isnan(v_3by3)):
                v_avg = np.nanmean(v_3by3)
            else:
                v_avg = np.nan
                
            verr_avg, verr_3by3 = verr_grid.value_at_coords(x, y, window=3, return_window=True)
            verr_3by3[verr_3by3 < -9998] = np.nan   # ITSLIVE nodata = -32767
            if np.any(~np.isnan(verr_3by3)):
                verr_avg = np.nanmean(verr_3by3)
            else:
                verr_avg = np.nan
            
            sampled2.append([v_avg, verr_avg])
            

    sampled = np.array(sampled)
    sampled2 = np.array(sampled2)
    # print(sampled)
    # print(row.Vx, float(df.loc[idx, 'SAV-peak-x']), float(df.loc[idx, 'SAV-peak-y']), sampled)
    
    df.loc[idx, 'pt0_vxavg'] = sampled[0, 1]
    df.loc[idx, 'pt0_vxgps'] = np.abs(gps.loc[0, 'vx (m/yr)'])
    df.loc[idx, 'pt0_vyavg'] = sampled[0, 3]
    df.loc[idx, 'pt0_vygps'] = np.abs(gps.loc[0, 'vy (m/yr)'])
    df.loc[idx, 'pt1_vxavg'] = sampled[1, 1]
    df.loc[idx, 'pt1_vxgps'] = np.abs(gps.loc[1, 'vx (m/yr)'])
    df.loc[idx, 'pt1_vyavg'] = sampled[1, 3]
    df.loc[idx, 'pt1_vygps'] = np.abs(gps.loc[1, 'vy (m/yr)'])
    df.loc[idx, 'pt2_vxavg'] = sampled[2, 1]
    df.loc[idx, 'pt2_vxgps'] = np.abs(gps.loc[2, 'vx (m/yr)'])
    df.loc[idx, 'pt2_vyavg'] = sampled[2, 3]
    df.loc[idx, 'pt2_vygps'] = np.abs(gps.loc[2, 'vy (m/yr)'])
    
    df.loc[idx, 'pt0_vxdiff'] = sampled[0, 0] - gps.loc[0, 'vx (m/yr)']
    df.loc[idx, 'pt0_vxavgdiff'] = sampled[0, 1] - gps.loc[0, 'vx (m/yr)']
    df.loc[idx, 'pt0_vydiff'] = sampled[0, 2] - gps.loc[0, 'vy (m/yr)']
    df.loc[idx, 'pt0_vyavgdiff'] = sampled[0, 3] - gps.loc[0, 'vy (m/yr)']
    df.loc[idx, 'pt1_vxdiff'] = sampled[1, 0] - gps.loc[1, 'vx (m/yr)']
    df.loc[idx, 'pt1_vxavgdiff'] = sampled[1, 1] - gps.loc[1, 'vx (m/yr)']
    df.loc[idx, 'pt1_vydiff'] = sampled[1, 2] - gps.loc[1, 'vy (m/yr)']
    df.loc[idx, 'pt1_vyavgdiff'] = sampled[1, 3] - gps.loc[1, 'vy (m/yr)']
    df.loc[idx, 'pt2_vxdiff'] = sampled[2, 0] - gps.loc[2, 'vx (m/yr)']
    df.loc[idx, 'pt2_vxavgdiff'] = sampled[2, 1] - gps.loc[2, 'vx (m/yr)']
    df.loc[idx, 'pt2_vydiff'] = sampled[2, 2] - gps.loc[2, 'vy (m/yr)']
    df.loc[idx, 'pt2_vyavgdiff'] = sampled[2, 3] - gps.loc[2, 'vy (m/yr)']
    
    df.loc[idx, 'pt0_vavg'] = sampled2[0, 0]
    df.loc[idx, 'pt0_vgps'] = gps.loc[0, 'v (m/yr)']
    df.loc[idx, 'pt0_vdiff'] = sampled2[0, 0] - gps.loc[0, 'v (m/yr)']
    df.loc[idx, 'pt0_verr'] = sampled2[0, 1]
    df.loc[idx, 'pt1_vavg'] = sampled2[1, 0]
    df.loc[idx, 'pt1_vgps'] = gps.loc[1, 'v (m/yr)']
    df.loc[idx, 'pt1_vdiff'] = sampled2[1, 0] - gps.loc[1, 'v (m/yr)']
    df.loc[idx, 'pt1_verr'] = sampled2[1, 1]  
    df.loc[idx, 'pt2_vavg'] = sampled2[2, 0]
    df.loc[idx, 'pt2_vgps'] = gps.loc[2, 'v (m/yr)']
    df.loc[idx, 'pt2_vdiff'] = sampled2[2, 0] - gps.loc[2, 'v (m/yr)']
    df.loc[idx, 'pt2_verr'] = sampled2[2, 1]  
    


In [7]:
gps

,Unnamed: 0,date1,date2,start_easting,start_northing,end_easting,end_northing,distance_traveled (m),velocity (m/d),start_easting_3413,start_northing_3413,end_easting_3413,end_northing_3413,geometry,vx (m/yr),vy (m/yr),v (m/yr)
0,0,2018-10-05,2018-09-30,621383.084841,6.738920e+06,621381.738315,6.738918e+06,2.015470,-0.403094,-3.227459e+06,212767.842312,-3.227460e+06,212769.385111,POINT (-3227460.401 212769.385),102.707969,-112.624285,147.129311
1,1,2018-10-05,2018-09-30,610531.630118,6.737073e+06,610530.195434,6.737074e+06,1.703303,-0.340661,-3.228240e+06,224145.144740,-3.228239e+06,224146.528571,POINT (-3228239.026 224146.529),-79.947760,-101.019600,124.341145
2,2,2018-10-05,2018-09-30,601810.429736,6.733774e+06,601809.124834,6.733773e+06,1.881326,-0.376265,-3.230736e+06,233478.821992,-3.230737e+06,233480.308331,POINT (-3230737.265 233480.308),92.088135,-108.502758,137.336804


In [8]:
df
# gps

,Label,Start date,End date,Duration (days),Template size (px),Template size (m),Pixel spacing (px),Pixel spacing (m),Prefilter,Subpixel,...,pt0_vdiff,pt0_verr,pt1_vavg,pt1_vgps,pt1_vdiff,pt1_verr,pt2_vavg,pt2_vgps,pt2_vdiff,pt2_verr
0,LS8-20180304-20180405,20180304,20180405,32,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,34.784609,40.000000,126.000000,135.694853,-9.694853,37.555557,151.625000,NaN,NaN,41.500000
1,LS8-20180405-20180421,20180405,20180421,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,-126.097831,118.222221,143.777771,NaN,NaN,75.333336,NaN,NaN,NaN,NaN
2,LS8-20180421-20180523,20180421,20180523,32,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LS8-20180523-20180608,20180523,20180608,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,NaN,70.875000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LS8-20180412-20180428,20180412,20180428,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,-60.811069,120.444443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LS8-20180428-20180802,20180428,20180802,96,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LS8-20180802-20180818,20180802,20180818,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,-150.839183,46.444443,165.666672,198.998533,-33.331862,45.888889,73.500000,NaN,NaN,47.333332
7,LS8-20180818-20180903,20180818,20180903,16,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,-79.030334,48.444443,213.000000,203.338758,9.661242,48.333332,NaN,NaN,NaN,NaN
8,LS8-20180903-20181005,20180903,20181005,32,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,-27.823042,32.000000,197.555557,139.352737,58.202820,36.111111,102.333336,143.208386,-40.875050,31.111111
9,Sen2-20180306-20180316,20180306,20180316,10,NaN,varying (240-480),NaN,120,ITS_LIVE (5x5 Wallis operator),ITS_LIVE (16-node oversampling),...,NaN,95.000000,111.444443,121.110993,-9.666550,64.444443,354.000000,134.068561,219.931439,99.111115


In [9]:
df = df.replace(-np.inf, np.nan)
df = df.replace(np.inf, np.nan)
df.to_csv('../results_ITSLIVE.csv', index=False)

In [10]:
gps

,Unnamed: 0,date1,date2,start_easting,start_northing,end_easting,end_northing,distance_traveled (m),velocity (m/d),start_easting_3413,start_northing_3413,end_easting_3413,end_northing_3413,geometry,vx (m/yr),vy (m/yr),v (m/yr)
0,0,2018-10-05,2018-09-30,621383.084841,6.738920e+06,621381.738315,6.738918e+06,2.015470,-0.403094,-3.227459e+06,212767.842312,-3.227460e+06,212769.385111,POINT (-3227460.401 212769.385),102.707969,-112.624285,147.129311
1,1,2018-10-05,2018-09-30,610531.630118,6.737073e+06,610530.195434,6.737074e+06,1.703303,-0.340661,-3.228240e+06,224145.144740,-3.228239e+06,224146.528571,POINT (-3228239.026 224146.529),-79.947760,-101.019600,124.341145
2,2,2018-10-05,2018-09-30,601810.429736,6.733774e+06,601809.124834,6.733773e+06,1.881326,-0.376265,-3.230736e+06,233478.821992,-3.230737e+06,233480.308331,POINT (-3230737.265 233480.308),92.088135,-108.502758,137.336804


In [11]:
gps_xy

[(-3227460.40093354, 212769.38511073),
 (-3228239.02608854, 224146.52857051),
 (-3230737.26454084, 233480.30833112)]